In [1]:
import yfinance as yf 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
MSCI = "CW8.PA" #Amundi MSCI World Swap UCITS ETF EUR Acc
risk_free_annual = 0
Start_date = "2010-01-01"
End_date = None


In [3]:
def annualized_return(daily_returns):
    compounded = np.prod(1 + daily_returns)
    n_years = len(daily_returns) / 252.0
    return compounded**(1/n_years) - 1

In [4]:
def annualized_volatility(daily_returns):  
    return daily_returns.std() * np.sqrt(252)

In [5]:
def sharpe_ratio(daily_returns, rf_annual=0.0):
    rf_daily = (1 + rf_annual)**(1/252) - 1
    excess = daily_returns - rf_daily
    return np.sqrt(252) * excess.mean() / excess.std()

In [6]:
def max_drawdown(cum_returns):
    peak = np.maximum.accumulate(cum_returns)
    dd = (cum_returns - peak) / peak
    return dd.min()

In [7]:
def compute_metrics(price_series):
    returns = price_series.pct_change().dropna()
    cagr = annualized_return(returns.values)
    vol = annualized_volatility(returns.values)
    sr = sharpe_ratio(returns.values, risk_free_annual)
    wealth = ((1 + returns).cumprod())*100
    mdd = max_drawdown(wealth.values)
    return {"cagr":cagr, "vol":vol, "sharpe":sr, "max_drawdown":mdd,
            "returns":returns, "wealth":wealth}

In [ ]:
data = yf.download(MSCI, start=Start_date, end=End_date, progress=False)['Close'].dropna()
base = data

r_base = base.pct_change().dropna()
r_2x = 2 * r_base
lever = (1 + r_2x).cumprod() * base.iloc[0]

df = pd.concat([base, lever], axis=1)
df.columns = ["base", "lever"]

metrics_base = compute_metrics(df["base"])
metrics_lever = compute_metrics(df["lever"])

print("=== Metrics summary ===")
print("Base (MSCI World ETF):")
for k,v in metrics_base.items():
    if k in ["returns","wealth"]: continue
    print(f"  {k}: {v:.4f}")
print("\nLETF x2 (simulé):")
for k,v in metrics_lever.items():
    if k in ["returns","wealth"]: continue
    print(f"  {k}: {v:.4f}")

YF.download() has changed argument auto_adjust default to True



1 Failed download:
['CW8.PA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


IndexError: single positional indexer is out-of-bounds

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np


MSCI = "CW8.PA" 
Start_date = "2010-01-01"
End_date = "2025-10-01"
leverage = 2
annual_cost = 0.0038   
financing_cost = 0.0   
trading_days = 252


def annualized_return(returns):
    cum_ret = np.prod(1 + returns)
    n_years = len(returns) / trading_days
    return cum_ret ** (1 / n_years) - 1

def annualized_volatility(returns):
    return np.std(returns) * np.sqrt(trading_days)

def sharpe_ratio(returns, rf_annual=0):
    mean_excess = np.mean(returns) * trading_days - rf_annual
    vol = annualized_volatility(returns)
    return mean_excess / vol if vol != 0 else np.nan

def max_drawdown(wealth):
    cummax = np.maximum.accumulate(wealth)
    dd = (wealth - cummax) / cummax
    return np.min(dd)

def compute_metrics(price_series, rf_annual=0):
    returns = price_series.pct_change().dropna()
    wealth = (1 + returns).cumprod() * 100
    return {
        "cagr": annualized_return(returns),
        "vol": annualized_volatility(returns),
        "sharpe": sharpe_ratio(returns, rf_annual),
        "max_drawdown": max_drawdown(wealth.values),
        "returns": returns,
        "wealth": wealth
    }


base = yf.download(MSCI, start=Start_date, end=End_date, progress=False)['Close'].dropna()


r_base = base.pct_change().dropna()


daily_cost = (annual_cost + financing_cost) / trading_days
r_lever = leverage * r_base - daily_cost


wealth_base = (1 + r_base).cumprod() * 100
wealth_lever = (1 + r_lever).cumprod() * 100


df = pd.concat([wealth_base, wealth_lever], axis=1)
df.columns = ["base", "lever"]

metrics_base = compute_metrics(wealth_base)
metrics_lever = compute_metrics(wealth_lever)

print("=== Metrics summary ===")
print("Base (MSCI World ETF):")
for k,v in metrics_base.items():
    if k in ["returns","wealth"]: continue
    print(f"  {k}: {v:.4f}")

print("\nLETF x2 (simulé):")
for k,v in metrics_lever.items():
    if k in ["returns","wealth"]: continue
    print(f"  {k}: {v:.4f}")


1 Failed download:
['CW8.PA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
c:\Users\sosof\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


ZeroDivisionError: float division by zero

: 

NameError: name 'metrics_base' is not defined

In [ ]:
r_base

Ticker,CW8.PA
Date,


In [ ]:
plt.figure(figsize=(10,5))
plt.plot(metrics_base["wealth"], label="Amundi MSCI World Swap UCITS ETF EUR Acc")
plt.plot(metrics_lever["wealth"], label="Leveraged x2 Amundi MSCI World Swap UCITS ETF EUR Acc")
plt.title("Wealth index: 1 unit initial capital")
plt.legend()
plt.show()


NameError: name 'metrics_base' is not defined

<Figure size 1000x500 with 0 Axes>

In [9]:
def drawdown_series(wealth):
    peak = np.maximum.accumulate(wealth)
    return (wealth - peak) / peak

plt.figure(figsize=(10,4))
plt.plot(drawdown_series(metrics_base["wealth"]), label="X1 drawdown")
plt.plot(drawdown_series(metrics_lever["wealth"]), label="X2 drawdown")
plt.title("Drawdowns")
plt.legend()
plt.show()

NameError: name 'metrics_base' is not defined

<Figure size 1000x400 with 0 Axes>